# Motion Planning

This notebook introduces our lab's motion planning tools for robot manipulation. The system is built on the [Klampt (Klamp't) library](http://motion.cs.illinois.edu/software/klampt/latest/pyklampt_docs/), which provides powerful motion planning, kinematics, and simulation capabilities.

If you want to make full use of this motion planner or extend it for your specific application, it's highly recommended to become familiar with Klampt. Refer to the official documentation:
http://motion.cs.illinois.edu/software/klampt/latest/pyklampt_docs/

> ``NOTE: The motion planner visualization might not work in the notebook for you. If it doesn't work, you can run the code in a python script alongside reading the notebook.``


In [1]:
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
%pip install -q mediapy

Reading package lists... Done
E: Could not open lock file /var/lib/apt/lists/lock - open (13: Permission denied)
E: Unable to lock directory /var/lib/apt/lists/
W: Problem unlinking the file /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permission denied)
W: Problem unlinking the file /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permission denied)
Note: you may need to restart the kernel to use updated packages.


In [1]:
%load_ext autoreload
%autoreload 2

First, let's initialize our motion planner:

In [1]:
import numpy as np
import time

In [2]:
from clair_robotics_stack.planning.motion.motion_planner import MotionPlanner
from clair_robotics_stack.planning.motion.abstract_motion_planner import AbstractMotionPlanner

mp = MotionPlanner()

RobParser: Reading robot file /home/clair01/PycharmProjects/clair-robotics-stak-REPACKAGED/clair_robotics_stack/motion_planning/ur5e_rob/ur5e_hires.rob...
RobParser:    Parsing robot file, 8 links read...
RobParser: Loaded geometries in time 0.009457s, 11028 total primitive elements
RobParser: Done loading robot file /home/clair01/PycharmProjects/clair-robotics-stak-REPACKAGED/clair_robotics_stack/motion_planning/ur5e_rob/ur5e_hires.rob
RobParser: Reading robot file /home/clair01/PycharmProjects/clair-robotics-stak-REPACKAGED/clair_robotics_stack/motion_planning/ur5e_rob/ur5e_hires.rob...
RobParser:    Parsing robot file, 8 links read...
RobParser: Loaded geometries in time 0.025944s, 11028 total primitive elements
RobParser: Done loading robot file /home/clair01/PycharmProjects/clair-robotics-stak-REPACKAGED/clair_robotics_stack/motion_planning/ur5e_rob/ur5e_hires.rob


Now we'll visualize the motion planning environment. This gives us a 3D view of the robot(s) and workspace.
You can use the mouse to rotate, shift + mouse to zoom in/out, and alt + mouse to pan the view.

In [3]:
mp.visualize()
time.sleep(10)  # so other cells won't run if you press run all (which you shouldn't)

*** klampt.vis: using GLUT as the visualization backend ***
***      Some functionality may not be available!       ***
First show(), window title Klamp't visualizer (/home/clair01/PycharmProjects/clair-robotics-stack/.venv/lib/python3.10/site-packages/ipykernel_launcher.py)
GLUTWindowManager.show(): first window shown, starting the visualization thread
GLUTWindowManager.run_app_thread: creating window with name Klamp't visualizer (/home/clair01/PycharmProjects/clair-robotics-stack/.venv/lib/python3.10/site-packages/ipykernel_launcher.py) and status shown
GLUTBackend.createWindow Klamp't visualizer (/home/clair01/PycharmProjects/clair-robotics-stack/.venv/lib/python3.10/site-packages/ipykernel_launcher.py)
Windows ["Klamp't visualizer (/home/clair01/PycharmProjects/clair-robotics-stack/.venv/lib/python3.10/site-packages/ipykernel_launcher.py)"]
GLUTBackend: run with windows ["Klamp't visualizer (/home/clair01/PycharmProjects/clair-robotics-stack/.venv/lib/python3.10/site-packages/ipyke

/home/clair01/PycharmProjects/clair-robotics-stack/.venv/lib/python3.10/site-packages/klampt/vis/visualization.py:704: UserWarning: klampt.visualization: QT is not available, falling back to poorer
GLUT interface.  Returning to another GLUT thread will not work
properly.
  warnings.warn("klampt.visualization: QT is not available, falling back to poorer\nGLUT interface.  Returning to another GLUT thread will not work\nproperly.")


## Working with Robot Names

In our motion planning environment, we need to specify which robot we're working with for every operation. The environment can contain multiple robots, and each must be referenced by name.

By default, our environment has two robots named `"ur5e_1"` and `"ur5e_2"`. You can see all available robot names using:

In [4]:
mp.robot_name_mapping.keys()

dict_keys(['ur5e_1', 'ur5e_2'])

## Robot Configuration

When working with multiple robots, it's important to keep the motion planner updated with the current configuration of each robot. The `update_robot_config` method updates the robot's configuration in the motion planner, so it will consider the position of robot 1 for example when planning paths for robot 2.

In practice, you would typically retrieve the current configuration from the real robot using the robot interface (e.g., `robot.getActualQ()` as shown in the robot control notebook), and then update the motion planner with this configuration. In the next notebook, we'll introduce a more advanced interface that handles this synchronization for you automatically.

We can control the robots in our environment by updating their joint configurations. Let's set specific configurations for our robots. You will see the change in the visualization window that should have opened when you ran the `visualize` method.

In [5]:
mp.update_robot_config("ur5e_1", [np.pi/2, 0, -np.pi/2, 0, 0, 0])

In [6]:
mp.update_robot_config("ur5e_2", [0, -np.pi/4, 0, 0, 0, 0])

We can also visualize different configurations of the same robot with different colors. This is useful for debugging, seeing start and goal states for planning and what ever you need in your application. Note that if you don't provide a name for a config, it will be removed when vis_config is called again. You can provide name to a config as a function argument. rgba means red, green, blue, alpha (transparency).

In [7]:
mp.vis_config("ur5e_2", [-np.pi/2, -np.pi/2, 0, 0, 0, 0], rgba=[1, 0, 0, 0.5])
mp.vis_config("ur5e_2", [0, 0, 0, 0, 0, 0], rgba=[0, 1, 1, 0.5], vis_name="other_config")

## Collision Checking

A fundamental capability in motion planning is checking whether a configuration is feasible (collision-free). Let's test a configuration that causes collision.

In [8]:
collision_config = [0, 0, 0, 0, 0, 0]
mp.vis_config("ur5e_1", collision_config)
print(mp.is_config_feasible("ur5e_1", collision_config))

False


## Path Planning

Now let's try planning a path between two configurations. We'll first visualize the start and goal configurations.

In [9]:
q1 = [0.03923, -1.76654, -2.27161, -2.19254, 0.40798, 3.07572]
q2 = [-0.06798, -1.84265, -2.65451, 1.33179, 1.61574, 1.64959]

mp.vis_config("ur5e_1", q1, rgba=[1, 0, 0, 0.5], vis_name="start")
mp.vis_config("ur5e_1", q2, rgba=[0, 1, 0, 0.5], vis_name="goal")

The plan_from_start_to_goal_config method computes a collision-free path between the configurations.

In [10]:
path = mp.plan_from_start_to_goal_config("ur5e_1", q1, q2)

planning motion...
planning took  0.20786285400390625  seconds.


Let's examine the computed path. Each row represents a configuration along the path.

In [11]:
print(path)

[[0.03923, -1.76654, -2.27161, -2.19254, 0.40798, 3.07572], [1.4569609397711325, -1.1235774672454895, -0.431613626264431, 1.3838748964912675, -1.311742691211058, -1.2887792762460508], [-0.06798, -1.84265, -2.65451, 1.33179, 1.61574, 1.64959]]


In [12]:
for p in path:
    mp.update_robot_config("ur5e_1", p)
    time.sleep(1)

It may be useful for debugging to visualize the path. The motion planner provides a method to animate the path (you can see it's arguments for more options).
This way you can even debug things without moving the robots.

In [13]:
mp.animate_path("ur5e_1", path)
mp.animate_path("ur5e_1", path[::-1])

Note that we can't plan to or from configurations in collision. This will raise an exception!

In [14]:
#TODO improve this exception
path = mp.plan_from_start_to_goal_config("ur5e_1", q1, collision_config)

/home/clair01/PycharmProjects/clair-robotics-stack/.venv/lib/python3.10/site-packages/klampt/plan/robotplanning.py:252: UserWarning: Goal configuration fails ['obj collision 2 table2', 'obj collision 5 keep_out_from_ur3_zone']
  warnings.warn("Goal configuration fails {}".format(gfailures))


AttributeError: 'NoneType' object has no attribute 'space'

If you are not sure that both start and goal configurations are feasible, you can check them before planning, put the planning in a try-except block, or reimplement those methods to handle this case.

 > ``Note: The motion planner uses RRT as default with some default parameters. You can can look at init method and read KLAMPT libray documentation if you want ot change it``

 > ``NOTE: To save time, when the planning method is called, it first checks if there is a direct collision-free path (in configuration space) avialable from start to goal. If there is, it returns that path of lenght 2. Otherwise, it plans a path. In an envrionment without many obstacles, there are often such direct paths.``

## Robot Kinematics

The motion planner also provides forward and inverse kinematics functionality.
Recall that the URs have 6 joints, so the configuration is a 6D vector. Forward kinematics computes the end-effector pose given a configuration.

For pose/transform we use the KLAMPT library representation, which is a tuple of (rotation_matrix, translation_vector).
The rotation matrix is a 3x3 matrix flattened to a 9D vector, and the translation vector is a 3D vector.

 > ``IMPORTANT NOTE: The coordinates of transforms/points in the motion planner are in the world coordinate system/frame and not necessarily the robot's base frame.
 >  for example: the robot is at (1,1,0), aligned with the world coordinate system rotation. You compute forward kinematics and get a translation of (1,2,0) for the end effector, that's world coordinates. It would be (0,1,0) in the robot's base frame. The Robot interfaces on the other hand, always use the robot's base frame as there is no notion of world coordinates or other robots in the robot's interface.``

Here's an example of computing the forward kinematics for the first robot:


In [15]:
R_flat, t = mp.get_forward_kinematics("ur5e_1", [0, -np.pi/2, 0, -np.pi/2, 0, 0])
R = np.array(R_flat).reshape(3, 3)

print("Rotation matrix:")
print(R)
print("Translation vector:")
print(t)

Rotation matrix:
[[-1.00000000e+00  0.00000000e+00  1.79489988e-09]
 [-1.79489988e-09  0.00000000e+00 -1.00000000e+00]
 [ 0.00000000e+00 -1.00000000e+00  0.00000000e+00]]
Translation vector:
[1.7917044553320089e-10, -0.383172, 1.079229]


We can also compute the inverse kinematics to find a configuration that achieves a desired end-effector pose. Here we choose a goal position and an orientation where the end effector is pointing downwards.

In [16]:
goal_position = np.array([0, -0.5, 0.5])
goal_orientation_matrix = np.array([[1, 0, 0], [0, -1, 0], [0, 0, -1]])
goal_transform = (goal_orientation_matrix.flatten(), goal_position)
config = mp.ik_solve("ur5e_1", goal_transform)
print(config)
if config is not None:
    mp.update_robot_config("ur5e_1", config)

[1.3003001799472724, -1.5112575636413668, 1.0822593860482859, -1.1417993995447948, -1.5707961172280607, 2.871096486077436]


## Geometry and Transforms
The GeometryAndTransforms class provides helper methods for working with points, transforms, and geometry in the motion planning environment.
It is initialized with a MotionPlanner instance.
It can be used for example to transform points between robot and world coordinates. UR5e_2 is not at the origin. If we transforms it's origin to the world coordinates, we get the robot's base position.

In [17]:
from clair_robotics_stack.planning.motion.geometry_and_transforms import GeometryAndTransforms
gt = GeometryAndTransforms(mp)

ur5e_2_position = gt.point_robot_to_world("ur5e_2", [0,0,0])
print(ur5e_2_position)

[-0.76, -1.33, 0.0]


Let's use the motion planner visualization to show the point in the world. We can also use it to see that UR5e_1 is at the origin.

In [18]:
above_ur5e_2_position = ur5e_2_position.copy()
above_ur5e_2_position[2] += 1
mp.show_point_vis(above_ur5e_2_position, rgba=[0, 1, 0, 0.5])
mp.show_point_vis([0,0,1], name="above origin", rgba=[1, 0, 0, 0.5])

## Extending Motion Planner and Defining Custom Geometry
Have you been wondering where the geometry (positions and orientation of robots, tables, obstacles...) is defined to be as you saw it in the default motion planner we have been working with do far?
Most of the code for the motion planner is in the AbstractMotionPlanner file (ur_lav/motion_planning/abstract_motion_planner.py). The logic is implemented there. The Motion Planner we use subclasses it and provides the path to the klampt world file that defines the geometry and robots.

In [20]:
%pycat ../../motion_planning/motion_planner.py

As we will see, to define your own motion planner, all you have to do is subclass AbstractMotionPlanner and implement the methods that are abstract in the AbstractMotionPlanner. which are: _get_klampt_world_path and _add_attachments. For that you will have to write the klampt world file that defines the geometry and robots. It is simply an xml file. Here is an example of such a file:

In [21]:
%more ../../motion_planning/klampt_world.xml

The syntax and options for the xml file are documented in the KLAMPT library documentation:
http://motion.cs.illinois.edu/software/klampt/latest/pyklampt_docs/

Let's try and implement our own motion planner that uses a custom klampt world file.

### Cleaning up
We are going to create another motion planner. Since the visualization module of klampt is global, we need to do some clean up. This is just for this notebook. Generally, refrain from using more than one motion planner in the same run.


In [22]:
def destroy_mp_and_reset_vis(mp):
    '''
    Only for use here where we create multiple motion planners...
    '''

    AbstractMotionPlanner.vis_initialized = False
    from klampt import vis
    vis.clear()
    del mp

In [23]:
destroy_mp_and_reset_vis(mp)

## Creating a World File
Let's create a basic world file, with 3 robots this time. To make things interesting, one of them will be hang from the ceiling and the other two will be on a table.


In [24]:
example_world_file = """
<?xml version="1.0" encoding="UTF-8"?>
<world>

  <robot name="robot_ceiling" file="../../motion/ur5e_rob/ur5e_hires.rob" position="0 0 1.99" rotateRPY="0 3.14 0"/>
  <robot name="robot_table1" file="../../motion/ur5e_rob/ur5e_hires.rob" position="0.7 0.7 0.01" rotateRPY="0 0 0"/>
  <robot name="robot_table2" file="../../motion/ur5e_rob/ur5e_hires.rob" position="-0.7 -0.7 0.01" rotateRPY="0 0 0"/>


  <rigidObject name="ceiling"  position="0 0 2.0">
    <geometry mesh="../../motion/objects/cube.off" scale="2 2 0.01" translate="0 0 0" />
  </rigidObject>

   <rigidObject name="table"  position="0 0 0">
    <geometry mesh="../../motion/objects/cube.off" scale="2 2 0.01" translate="0 0 0" />
  </rigidObject>

</world>

"""

# save to file:
with open("example_klampt_world.xml", "w") as f:
    f.write(example_world_file)

```
  <rigidObject name="ceiling"  position="0 0 2.0">
    <geometry mesh="../../motion_planning/objects/cube.off" scale="2 2 0.01" translate="0 0 0" />
  </rigidObject>
```
creates a rigid object for the ceiling. A geometry is created from the ../../motion_planning/objects/cube.off mesh file, which is 1x1x1 cube. It is sclaed to 2x2x0.01 to be a thin plane and in the rigid object decleration it is translated by two meters up: position="0 0 2.0". A table plane is placed similarely.

```
  <robot name="robot_ceiling" file="../../motion_planning/ur5e_rob/ur5e_hires.rob" position="0 0 1.99" rotateRPY="0 3.14 0"/>
```
Uses a robot model that is placed in ../../motion_planning/ur5e_rob/ur5e_hires.rob. It puts it's bace in position 0 0 1.99 (ceiling) and rotates it 180 degrees (3.14 radians) around y axis to make it upside down.
Two other robots are placed on the table similarly.

## Creating a Motion Planner
Now that we have a file that defines the geometry, we can subclass the motion planner and create a new one that uses this file.
We are not adding attachments and make the trivial and minimal extension, we just return the path to the file we created above as world file, and we have a working motion planner, since all the logic for motion planning and the interfaces we saw above is implemented in the AbstractMotionPlanner class.


In [25]:
from clair_robotics_stack.planning.motion.abstract_motion_planner import AbstractMotionPlanner

class MyMotionPlanner(AbstractMotionPlanner):

    def _get_klampt_world_path(self):
        return "example_klampt_world.xml"

    def _add_attachments(self, robot, attachments):
        pass

In [26]:
my_mp = MyMotionPlanner()
my_mp.visualize()

 For example, we can update the configuration of the robot that is hanging from the ceiling. Note that we use the name we gave it in the world file.

In [27]:
my_mp.update_robot_config("robot_ceiling", [0, 0, 0, 0, 0, 0])

We can see that we have the three robots we defined in the environment:

In [28]:
my_mp.robot_name_mapping.keys()

dict_keys(['robot_ceiling', 'robot_table1', 'robot_table2'])